# 9.1 Extending Built-in Type

In [1]:
# File setwrapper.py
class Set:
    def __init__(self, value = []): # Constructor
        self.data = [] # Manages a list
        self.concat(value)
    def intersect(self, other): # other is any sequence
        res = [] # self is the subjecthttp://localhost:8888/notebooks/Python_Advanced/09_Advanced%20Class%20Topics.ipynb#
        for x in self.data:
            if x in other: # Pick common items
                res.append(x)
        return Set(res) # Return a new Set
    def union(self, other): # other is any sequence
        res = self.data[:] # Copy of my list
        for x in other: # Add items in other
            if not x in res:
                res.append(x)
        return Set(res)
    def concat(self, value): # value: list, Set...
        for x in value: # Removes duplicates
            if not x in self.data:
                self.data.append(x)
    def __len__(self): return len(self.data) # len(self), if self
    def __getitem__(self, key): return self.data[key] # self[i], self[i:j]
    def __and__(self, other): return self.intersect(other) # self & other
    def __or__(self, other): return self.union(other) # self | other
    def __repr__(self): return 'Set:' + repr(self.data) # print(self),...
    def __iter__(self): return iter(self.data) # for x in self,...

In [2]:
x = Set([1, 3, 5, 7])
print(x.union(Set([1, 4, 7]))) # prints Set:[1, 3, 5, 7, 4]
print(x | Set([1, 4, 6])) # prints Set:[1, 3, 5, 7, 4, 6]

Set:[1, 3, 5, 7, 4]
Set:[1, 3, 5, 7, 4, 6]


## Entending Types by Subclassing

In [3]:
# File typesubclass.py
class MyList(list):
    def __getitem__(self, offset):
        print('(indexing %s at %s)' % (self, offset))
        return list.__getitem__(self, offset - 1)
if __name__ == '__main__':
    print(list('abc'))
    x = MyList('abc') # __init__ inherited from list
    print(x) # __repr__ inherited from list
    print(x[1]) # MyList.__getitem__
    print(x[3]) # Customizes list superclass method
    x.append('spam'); print(x) # Attributes from list superclass
    x.reverse(); print(x)

['a', 'b', 'c']
['a', 'b', 'c']
(indexing ['a', 'b', 'c'] at 1)
a
(indexing ['a', 'b', 'c'] at 3)
c
['a', 'b', 'c', 'spam']
['spam', 'c', 'b', 'a']


In [4]:
# File setsubclass.py
class Set(list):
    def __init__(self, value = []): # Constructor
        list.__init__([]) # Customizes list
        self.concat(value) # Copies mutable defaults
    def intersect(self, other): # other is any sequence
        res = [] # self is the subject
        for x in self:
            if x in other: # Pick common items
                res.append(x)
        return Set(res) # Return a new Set
    def union(self, other): # other is any sequence
        res = Set(self) # Copy me and my list
        res.concat(other)
        return res
    def concat(self, value): # value: list, Set, etc.
        for x in value: # Removes duplicates
            if not x in self:
                self.append(x)
    def __and__(self, other): return self.intersect(other)
    def __or__(self, other): return self.union(other)
    def __repr__(self): return 'Set:' + list.__repr__(self)
if __name__ == '__main__':
    x = Set([1,3,5,7])
    y = Set([2,1,4,5,6])
    print(x, y, len(x))
    print("-"*40)
    print(x.intersect(y), y.union(x))
    print("-"*40)
    print(x & y, x | y)
    print("-"*40)
    x.reverse(); print(x)

Set:[1, 3, 5, 7] Set:[2, 1, 4, 5, 6] 4
----------------------------------------
Set:[1, 5] Set:[2, 1, 4, 5, 6, 3, 7]
----------------------------------------
Set:[1, 5] Set:[1, 3, 5, 7, 2, 4, 6]
----------------------------------------
Set:[7, 5, 3, 1]


# 9.2 Class model new style

Class kiểu mới trong python 3 có object.
```python
class C(object):
    data = 'spam'
    def __getattr__(self, name):
        print(name)
        return getattr(self.data, name)
X = C()
X[0]
```
`Output: TypeError: 'C' object does not support indexing`

In [5]:
class C(object): pass 
X = C()
X.normal = lambda: 99
X.normal() # Normals still from instance

99

In [6]:
X.__add__ =lambda y: 88 + y

In [7]:
X.__add__(1)

89

Cộng trực tiếp sẽ không được.
```python
X+1
```
`Output: TypeError: unsupported operand type(s) for +: 'C' and 'int'`

In [8]:
class C(object):
    def __getattr__(self, name): 
        print(name)

In [9]:
X = C()
X.normal

normal


In [10]:
 X.__add__ # Direct calls by name are too, but expressions are not

__add__


Cộng trực tiếp sẽ gây ra lỗi.
```python
X+1
```
`Output: TypeError: unsupported operand type(s) for +: 'C' and 'int'`

In [11]:
class C(object):
    data = 'spam'
    def __getattr__(self, name):
        print('getattr: ' + name)
        return getattr(self.data, name)

In [12]:
X = C()
X.__getitem__(1)

getattr: __getitem__


'p'

Lấy index gây ra lỗi.
```python
X[1]
```
`Output: TypeError: 'C' object does not support indexing`

In [13]:
X.__add__('eggs')

getattr: __add__


'spameggs'

Cộng trực tiếp gây ra lỗi.
```python
 X + 'eggs'
```
`Output: TypeError: unsupported operand type(s) for +: 'C' and 'str'`

Thử type dùng phương thức.
```python
type(X).__add__(X, 'eggs')
```
`Output: AttributeError: type object 'C' has no attribute '__add__'`

In [14]:
class C(object): 
    data = 'spam'
    def __getattr__(self, name): # Catch normal names
        print('getattr: ' + name)
        return getattr(self.data, name)
    def __getitem__(self, i): # Redefine built-ins
        print('getitem: ' + str(i))
        return self.data[i] # Run expr or getattr
    def __add__(self, other):
        print('add: ' + other)
        return getattr(self.data, '__add__')(other)
X=C()
X.upper

getattr: upper


<function str.upper()>

In [15]:
X.upper()  

getattr: upper


'SPAM'

In [16]:
X[1]

getitem: 1


'p'

In [17]:
 X.__getitem__(1)

getitem: 1


'p'

In [18]:
type(X).__getitem__(X, 1)

getitem: 1


'p'

In [19]:
 X + 'eggs'

add: eggs


'spameggs'

In [20]:
X.__add__('eggs')

add: eggs


'spameggs'

In [21]:
type(X).__add__(X, 'eggs')

add: eggs


'spameggs'

# 9.3 Type model changes

In [22]:
class C: pass
I = C() 
type(I), I.__class__ 

(__main__.C, __main__.C)

In [23]:
type(C), C.__class__ # Class is a type, and type is a clas

(type, type)

In [24]:
type([1, 2, 3]), [1, 2, 3].__class__

(list, list)

In [25]:
type(list), list.__class__

(type, type)

In [26]:
class C: pass
class D: pass

In [27]:
c, d = C(), D()
type(c) == type(d)

False

In [28]:
type(c), type(d)

(__main__.C, __main__.D)

In [29]:
c.__class__, d.__class__

(__main__.C, __main__.D)

In [30]:
c1, c2 = C(), C()
type(c1) == type(c2)

True

## Class bắt nguồn từ object

In [31]:
class C: pass # For new-style classes
X = C()
type(X), type(C) # Type is class 

(__main__.C, type)

In [32]:
isinstance(X, object)

True

In [33]:
isinstance(C, object)

True

In [34]:
type('spam'), type(str)

(str, type)

In [35]:
isinstance('spam', object)

True

In [36]:
isinstance(str, object)

True

In [37]:
type(type) # All classes are types

type

In [38]:
type(object)

type

In [39]:
 isinstance(type, object) # All clases bắt nguồn từ object

True

In [40]:
isinstance(object, type) # Type make classes

True

In [41]:
type is object

False

In [42]:
dir(object)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [43]:
class C: pass # This means all classes get defaults in 3.X
C.__bases__

(object,)

In [44]:
C().__repr__

<method-wrapper '__repr__' of C object at 0x036587F0>

# 9.4 Diamod Inheritance change

In [45]:
class A(object): attr = 1 # New-style ("object" not required in 3.X)
class B(A): pass
class C(A): attr = 2
class D(B, C): pass

In [46]:
x = D()
x.attr

2

In [47]:
class A: attr = 1 # Classic
class B(A): pass
class C(A): attr = 2
class D(B, C): attr = C.attr

In [48]:
x = D()
x.attr

2

In [49]:
class A(object): attr = 1 # New-style
class B(A): pass
class C(A): attr = 2
class D(B, C): attr = B.attr

In [50]:
x = D()
x.attr

1

In [51]:
class A:
    def meth(s): print('A.meth')
class C(A):
    def meth(s): print('C.meth')
class B(A):
    pass
class D(B,C):
    pass
x = D() # Will vary per class type
x.meth()

C.meth


In [52]:
class D(B, C): meth = C.meth # <== Pick C's method: new-style (and 3.X)
x = D()
x.meth()

C.meth


In [53]:
class D(B, C): meth = B.meth # <== Pick B's method: classic
x = D()
x.meth()

A.meth


## Method Resolution Order (MRO)

In [54]:
class A: pass
class B(A): pass # Diamonds: order differs for newstyle
class C(A): pass # Breadth-first across lower levels
class D(B, C): pass
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [55]:
A.__bases__

(object,)

In [56]:
B.__bases__

(__main__.A,)

In [57]:
C.__bases__

(__main__.A,)

In [58]:
D.__bases__

(__main__.B, __main__.C)

In [59]:
class X: pass
class Y: pass
class A(X): pass # Nondiamond: depth first then left to right
class B(Y): pass # Though implied "object" always forms a diamond
class D(A, B): pass
D.mro()

[__main__.D, __main__.A, __main__.X, __main__.B, __main__.Y, object]

In [60]:
X.__bases__, Y.__bases__

((object,), (object,))

In [61]:
A.__bases__, B.__bases__

((__main__.X,), (__main__.Y,))

In [62]:
D.mro() == list(D.__mro__)

True

In [63]:
[cls.__name__ for cls in D.__mro__]

['D', 'A', 'X', 'B', 'Y', 'object']

In [64]:
# File mapattr.py
import pprint
def trace(X, label='', end='\n'):
    print(label + pprint.pformat(X) + end) # Print nicely
def filterdictvals(D, V):
    """
    dict D with entries for value V removed.
    filterdictvals(dict(a=1, b=2, c=1), 1) => {'b': 2}
    """
    return {K: V2 for (K, V2) in D.items() if V2 != V}
def invertdict(D):
    """
    dict D with values changed to keys (grouped by values).
    Values must all be hashable to work as dict/set keys.
    invertdict(dict(a=1, b=2, c=1)) => {1: ['a', 'c'], 2: ['b']}
    """
    def keysof(V):
        return sorted(K for K in D.keys() if D[K] == V)
    return {V: keysof(V) for V in set(D.values())}
def dflr(cls):
    """
    Classic depth-first left-to-right order of class tree at cls.
    Cycles not possible: Python disallows on __bases__ changes.
    """
    here = [cls]
    for sup in cls.__bases__:
        here += dflr(sup)
    return here
def inheritance(instance):
    """
    Inheritance order sequence: new-style (MRO) or classic (DFLR)
    """
    if hasattr(instance.__class__, '__mro__'):
        return (instance,) + instance.__class__.__mro__
    else:
        return [instance] + dflr(instance.__class__)
def mapattrs(instance, withobject=False, bysource=False):
    """
    dict with keys giving all inherited attributes of instance,
    with values giving the object that each is inherited from.
    withobject: False=remove object built-in class attributes.
    bysource: True=group result by objects instead of attributes.
    Supports classes with slots that preclude __dict__ in instances.
    """
    attr2obj = {}
    inherits = inheritance(instance)
    for attr in dir(instance):
        for obj in inherits:
            if hasattr(obj, '__dict__') and attr in obj.__dict__: # See slots
                attr2obj[attr] = obj
                break
    if not withobject:
        attr2obj = filterdictvals(attr2obj, object)
    return attr2obj if not bysource else invertdict(attr2obj)
if __name__ == '__main__':
    print('Classic classes in 2.X, new-style in 3.X')
    class A: attr1 = 1
    class B(A): attr2 = 2
    class C(A): attr1 = 3
    class D(B, C): pass
    I = D()
    print('Py=>%s' % I.attr1) # Python's search == ours?
    trace(inheritance(I), 'INH\n') # [Inheritance order]
    trace(mapattrs(I), 'ATTRS\n') # Attrs => Source
    trace(mapattrs(I, bysource=True), 'OBJS\n') # Source => [Attrs]
    print('New-style classes in 2.X and 3.X')
    class A(object): attr1 = 1 # "(object)" optional in 3.X
    class B(A): attr2 = 2
    class C(A): attr1 = 3
    class D(B, C): pass
    I = D()
    print('Py=>%s' % I.attr1)
    trace(inheritance(I), 'INH\n')
    trace(mapattrs(I), 'ATTRS\n')
    trace(mapattrs(I, bysource=True), 'OBJS\n')

Classic classes in 2.X, new-style in 3.X
Py=>3
INH
(<__main__.D object at 0x03661E90>,
 <class '__main__.D'>,
 <class '__main__.B'>,
 <class '__main__.C'>,
 <class '__main__.A'>,
 <class 'object'>)

ATTRS
{'__dict__': <class '__main__.A'>,
 '__doc__': <class '__main__.D'>,
 '__module__': <class '__main__.D'>,
 '__weakref__': <class '__main__.A'>,
 'attr1': <class '__main__.C'>,
 'attr2': <class '__main__.B'>}

OBJS
{<class '__main__.A'>: ['__dict__', '__weakref__'],
 <class '__main__.B'>: ['attr2'],
 <class '__main__.C'>: ['attr1'],
 <class '__main__.D'>: ['__doc__', '__module__']}

New-style classes in 2.X and 3.X
Py=>3
INH
(<__main__.D object at 0x03682D30>,
 <class '__main__.D'>,
 <class '__main__.B'>,
 <class '__main__.C'>,
 <class '__main__.A'>,
 <class 'object'>)

ATTRS
{'__dict__': <class '__main__.A'>,
 '__doc__': <class '__main__.D'>,
 '__module__': <class '__main__.D'>,
 '__weakref__': <class '__main__.A'>,
 'attr1': <class '__main__.C'>,
 'attr2': <class '__main__.B'>}

OBJS

In [65]:
from mapattrs import trace,dflr,inheritance,mapattrs
from testmixin0 import Sub
I = Sub() # Sub inherits from Super and ListInstance roots
trace(dflr(I.__class__))

[<class 'testmixin0.Sub'>,
 <class 'testmixin0.Super'>,
 <class 'object'>,
 <class 'listinstance.ListInstance'>,
 <class 'object'>]



In [66]:
trace(inheritance(I))

(<testmixin0.Sub object at 0x0366A350>,
 <class 'testmixin0.Sub'>,
 <class 'testmixin0.Super'>,
 <class 'listinstance.ListInstance'>,
 <class 'object'>)



In [67]:
trace(mapattrs(I))

{'_ListInstance__attrnames': <class 'listinstance.ListInstance'>,
 '__dict__': <class 'testmixin0.Super'>,
 '__doc__': <class 'testmixin0.Sub'>,
 '__init__': <class 'testmixin0.Sub'>,
 '__module__': <class 'testmixin0.Sub'>,
 '__str__': <class 'listinstance.ListInstance'>,
 '__weakref__': <class 'testmixin0.Super'>,
 'data1': <testmixin0.Sub object at 0x0366A350>,
 'data2': <testmixin0.Sub object at 0x0366A350>,
 'data3': <testmixin0.Sub object at 0x0366A350>,
 'ham': <class 'testmixin0.Super'>,
 'spam': <class 'testmixin0.Sub'>}



In [68]:
trace(mapattrs(I, bysource=True))

{<testmixin0.Sub object at 0x0366A350>: ['data1', 'data2', 'data3'],
 <class 'listinstance.ListInstance'>: ['_ListInstance__attrnames', '__str__'],
 <class 'testmixin0.Super'>: ['__dict__', '__weakref__', 'ham'],
 <class 'testmixin0.Sub'>: ['__doc__', '__init__', '__module__', 'spam']}



In [69]:
trace(mapattrs(I, withobject=True))

{'_ListInstance__attrnames': <class 'listinstance.ListInstance'>,
 '__class__': <class 'object'>,
 '__delattr__': <class 'object'>,
 '__dict__': <class 'testmixin0.Super'>,
 '__dir__': <class 'object'>,
 '__doc__': <class 'testmixin0.Sub'>,
 '__eq__': <class 'object'>,
 '__format__': <class 'object'>,
 '__ge__': <class 'object'>,
 '__getattribute__': <class 'object'>,
 '__gt__': <class 'object'>,
 '__hash__': <class 'object'>,
 '__init__': <class 'testmixin0.Sub'>,
 '__init_subclass__': <class 'object'>,
 '__le__': <class 'object'>,
 '__lt__': <class 'object'>,
 '__module__': <class 'testmixin0.Sub'>,
 '__ne__': <class 'object'>,
 '__new__': <class 'object'>,
 '__reduce__': <class 'object'>,
 '__reduce_ex__': <class 'object'>,
 '__repr__': <class 'object'>,
 '__setattr__': <class 'object'>,
 '__sizeof__': <class 'object'>,
 '__str__': <class 'listinstance.ListInstance'>,
 '__subclasshook__': <class 'object'>,
 '__weakref__': <class 'testmixin0.Super'>,
 'data1': <testmixin0.Sub object 

In [70]:
amap = mapattrs(I, withobject=True, bysource=True)
trace(amap)

{<testmixin0.Sub object at 0x0366A350>: ['data1', 'data2', 'data3'],
 <class 'listinstance.ListInstance'>: ['_ListInstance__attrnames', '__str__'],
 <class 'testmixin0.Super'>: ['__dict__', '__weakref__', 'ham'],
 <class 'testmixin0.Sub'>: ['__doc__', '__init__', '__module__', 'spam'],
 <class 'object'>: ['__class__',
                    '__delattr__',
                    '__dir__',
                    '__eq__',
                    '__format__',
                    '__ge__',
                    '__getattribute__',
                    '__gt__',
                    '__hash__',
                    '__init_subclass__',
                    '__le__',
                    '__lt__',
                    '__ne__',
                    '__new__',
                    '__reduce__',
                    '__reduce_ex__',
                    '__repr__',
                    '__setattr__',
                    '__sizeof__',
                    '__subclasshook__']}



In [71]:
# File mapattrs-slots.py
class A(object): __slots__ = ['a', 'b']; x = 1; y = 2
class B(A): __slots__ = ['b', 'c']
class C(A): x = 2
class D(B, C):
    z = 3
    def __init__(self): self.name = 'Bob';
        
I = D()
trace(mapattrs(I, bysource=True)) # Also: trace(mapattrs(I))

{<__main__.D object at 0x0367E7E0>: ['name'],
 <class '__main__.A'>: ['a', 'y'],
 <class '__main__.C'>: ['x'],
 <class '__main__.D'>: ['__dict__',
                        '__doc__',
                        '__init__',
                        '__module__',
                        '__weakref__',
                        'z'],
 <class '__main__.B'>: ['__slots__', 'b', 'c']}



## New-Style class extension
### Slots: Attribute Declarations

Chưa được khởi tạo sẽ báo lỗi.
```python
class limiter(object):
    __slots__ = ['age', 'name', 'job']
x = limiter()
x.age
```
`Output: AttributeError: age`

In [72]:
class limiter(object):
    __slots__ = ['age', 'name', 'job']
x = limiter()
x.age = 40 # Looks like instance data
x.age

40

Sai thuộc tính trong \_\_slot\_\_.
```python
x.ape = 1000
```
`Ouput: AttributeError: 'limiter' object has no attribute 'ape'`

In [73]:
class C: 
    __slots__ = ['a', 'b']

In [74]:
X = C()
X.a = 1
X.a

1

Nó sẽ báo lỗi với \_\_dict\_\_.
```python
X.__dict__
```
`Output: AttributeError: 'C' object has no attribute '__dict__'`


In [75]:
 getattr(X, 'a')

1

In [76]:
setattr(X, 'b', 2)
X.b

2

In [77]:
'a' in dir(X)

True

In [78]:
'b' in dir(X)

True

Nó báo lỗi khi khởi tạo bằng \_\_init\_\_.
```python
class D: # Use D(object) for same result in 2.X
    __slots__ = ['a', 'b']
    def __init__(self):
        self.d = 4 # Cannot add new names if no __dict__
X = D()
```
`Output: AttributeError: 'D' object has no attribute 'd'`

In [79]:
class D:
    __slots__ = ['a', 'b', '__dict__'] # Name __dict__ to include one too
    c = 3 # Class attrs work normally
    def __init__(self):
        self.d = 4 # d stored in __dict__
X = D()
X.d

4

In [80]:
X.c

3

In [81]:
X.a = 1
X.b = 2

In [82]:
X.__dict__

{'d': 4}

In [83]:
 X.__slots__

['a', 'b', '__dict__']

In [84]:
 getattr(X, 'a'), getattr(X, 'c'), getattr(X, 'd')

(1, 3, 4)

In [85]:
for attr in list(X.__dict__) + X.__slots__: # Wrong...
    print(attr, '=>', getattr(X, attr))

d => 4
a => 1
b => 2
__dict__ => {'d': 4}


In [86]:
for attr in list(getattr(X, '__dict__', [])) + getattr(X, '__slots__', []):
    print(attr, '=>', getattr(X, attr))

d => 4
a => 1
b => 2
__dict__ => {'d': 4}


In [87]:
class E:
    __slots__ = ['c', 'd'] # Superclass has slots
class D(E):
    __slots__ = ['a', '__dict__'] # But so does its subclass
X = D()
X.a = 1; X.b = 2; X.c = 3 # The instance is the union

In [88]:
X.a, X.c

(1, 3)

In [89]:
E.__slots__

['c', 'd']

In [90]:
D.__slots__

['a', '__dict__']

In [91]:
X.__slots__

['a', '__dict__']

In [92]:
X.__dict__

{'b': 2}

In [93]:
for attr in list(getattr(X, '__dict__', [])) + getattr(X, '__slots__', []):
    print(attr, '=>', getattr(X, attr))

b => 2
a => 1
__dict__ => {'b': 2}


In [94]:
dir(X)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 'a',
 'b',
 'c',
 'd']

In [95]:
class Slotful:
    __slots__ = ['a', 'b', '__dict__']
    def __init__(self, data):
        self.c = data
I = Slotful(3)
I.a, I.b = 1, 2
I.a, I.b, I.c

(1, 2, 3)

In [96]:
I.__dict__

{'c': 3}

In [97]:
[x for x in dir(I) if not x.startswith('__')]

['a', 'b', 'c']

In [98]:
I.__dict__['c']

3

In [99]:
getattr(I, 'c'), getattr(I, 'a')

(3, 1)

In [100]:
for a in (x for x in dir(I) if not x.startswith('__')):
    print(a, getattr(I, a))

a 1
b 2
c 3


In [101]:
class C: pass # Bullet 1: slots in sub but not super
class D(C): __slots__ = ['a'] # Makes instance dict for nonslots
X = D() # But slot name still managed in class
X.a = 1; X.b = 2

In [102]:
X.__dict__

{'b': 2}

In [103]:
 D.__dict__.keys()

dict_keys(['__module__', '__slots__', 'a', '__doc__'])

In [104]:
class C: __slots__ = ['a'] # Bullet 2: slots in super but not sub
class D(C): pass # Makes instance dict for nonslots
X = D() # But slot name still managed in class
X.a = 1; X.b = 2
X.__dict__

{'b': 2}

In [105]:
C.__dict__.keys()

dict_keys(['__module__', '__slots__', 'a', '__doc__'])

In [106]:
class C: __slots__ = ['a']
class D(C): __slots__ = ['a']

Lỗi khi xài attribute class trùng.
```python
class C: __slots__ = ['a']; a = 99 # Bullet 4: no class-level defaults
```
`Output: ValueError: 'a' in __slots__ conflicts with class variable`

In [107]:
class C: __slots__ = ['a'] # Assumes universal use, differing names
class D(C): __slots__ = ['b']

In [108]:
X = D()
X.a = 1; X.b = 2

Thực thể không có thuộc tính \_\_dict\_\_
```python
X.__dict__
```
`Output: AttributeError: 'D' object has no attribute '__dict__'`

In [109]:
C.__dict__.keys(), D.__dict__.keys()

(dict_keys(['__module__', '__slots__', 'a', '__doc__']),
 dict_keys(['__module__', '__slots__', 'b', '__doc__']))

In [110]:
# File slots-test.py
import timeit
base = """
Is = []
for i in range(1000):
    X = C()
    X.a = 1; X.b = 2; X.c = 3; X.d = 4
    t = X.a + X.b + X.c + X.d
    Is.append(X)
"""
stmt = """
class C:
    __slots__ = ['a', 'b', 'c', 'd']
""" + base
print('Slots =>', end=' ')
print(min(timeit.repeat(stmt, number=1000, repeat=3)))

stmt = """
class C:
    pass
""" + base
print('Nonslots=>', end=' ')
print(min(timeit.repeat(stmt, number=1000, repeat=3)))

Slots => 1.259286038
Nonslots=> 1.5272776159999992


# 9.5 Properties Attribute Accessors

In [111]:
class operators:
    def __getattr__(self, name):
        if name == 'age':
            return 40
        else:
            raise AttributeError(name)
x = operators()
x.age # Runs __getat

40

Nếu khác attribute age.
```python
x.name
```
`Output: AttributeError: name`


In [112]:
class properties(object): 
    def getage(self):
        return 40
    age = property(getage, None, None, None) 

In [113]:
x = properties()
x.age

40

Giống hàm \_\_getattr\_\_.
```python
x.name
```
`Output: AttributeError: 'properties' object has no attribute 'name'`

In [114]:
class properties(object):
    def getage(self):
        return 40
    def setage(self, value):
        print('set age: %s' % value)
        self._age = value
    age = property(getage, setage, None, None)

In [115]:
x = properties()
x.age

40

In [116]:
x.age = 42

set age: 42


In [117]:
 x._age

42

In [118]:
x.job = 'trainer'
x.job

'trainer'

In [119]:
class operators:
    def __getattr__(self, name): # On undefined reference
        if name == 'age':
            return 40
        else:
            raise AttributeError(name)
    def __setattr__(self, name, value): # On all assignments
        print('set: %s %s' % (name, value))
        if name == 'age':
            self.__dict__['_age'] = value # Or object.__setattr__()
        else:
            self.__dict__[name] = value

In [120]:
x = operators()
x.age

40

In [121]:
x.age = 41

set: age 41


In [122]:
x._age

41

In [123]:
 x.age

40

In [124]:
 x.job = 'trainer'

set: job trainer


In [125]:
 x.job

'trainer'

## Descriptor

In [126]:
class AgeDesc(object):
    def __get__(self, instance, owner): return 40
    def __set__(self, instance, value): instance._age = value
class descriptors(object):
    age = AgeDesc()

In [127]:
x = descriptors()
x.age

40

In [128]:
x.age = 42 # Runs AgeDesc.__set__
x._age

42

# 9.6 Static and class method

In [129]:
class Spam:
    numInstances = 0
    def __init__(self):
        Spam.numInstances = Spam.numInstances + 1
    def printNumInstances():
        print("Number of instances created: %s" % Spam.numInstances)

In [130]:
a = Spam() # Can call functions in class in 3.X
b = Spam() # Calls through instances still pass a self
c = Spam()

In [131]:
Spam.printNumInstances()

Number of instances created: 3


Lỗi khi thực thể truy xuất.
```python
a.printNumInstances()
```
`Output: TypeError: printNumInstances() takes 0 positional arguments but 1 was given`

In [132]:
def printNumInstances():
    print("Number of instances created: %s" % Spam.numInstances)
class Spam:
    numInstances = 0
    def __init__(self):
        Spam.numInstances = Spam.numInstances + 1

In [133]:
a = Spam()
b = Spam()
c = Spam()

In [134]:
printNumInstances()

Number of instances created: 3


In [135]:
Spam.numInstances

3

In [136]:
class Spam:
    numInstances = 0
    def __init__(self):
        Spam.numInstances = Spam.numInstances + 1
    def printNumInstances(self):
        print("Number of instances created: %s" % Spam.numInstances)

In [137]:
a, b, c = Spam(), Spam(), Spam()
a.printNumInstances()

Number of instances created: 3


In [138]:
Spam.printNumInstances(a)

Number of instances created: 3


In [139]:
Spam().printNumInstances()

Number of instances created: 4


In [140]:
class Methods:
    def imeth(self, x): # Normal instance method: passed a self
        print([self, x])
    def smeth(x): # Static: no instance passed
        print([x])
    def cmeth(cls, x): # Class: gets class, not instance
        print([cls, x])
    smeth = staticmethod(smeth) # Make smeth a static method (or @: ahead)
    cmeth = classmethod(cmeth) # Make cmeth a class method (or @: ahead)

In [141]:
obj = Methods() # Callable through instance or class
obj.imeth(1)

[<__main__.Methods object at 0x0368FB30>, 1]


In [142]:
Methods.imeth(obj, 2)

[<__main__.Methods object at 0x0368FB30>, 2]


In [143]:
Methods.smeth(3)

[3]


In [144]:
obj.smeth(4)

[4]


In [145]:
 Methods.cmeth(5)

[<class '__main__.Methods'>, 5]


In [146]:
obj.cmeth(6)

[<class '__main__.Methods'>, 6]


In [147]:
class Spam:
    numInstances = 0 # Use static method for class data
    def __init__(self):
        Spam.numInstances += 1
    def printNumInstances():
        print("Number of instances: %s" % Spam.numInstances)
    printNumInstances = staticmethod(printNumInstances)

In [148]:
a = Spam()
b = Spam()
c = Spam()
Spam.printNumInstances()

Number of instances: 3


In [149]:
a.printNumInstances()

Number of instances: 3


In [150]:
class Sub(Spam):
    def printNumInstances(): # Override a static method
        print("Extra stuff...") # But call back to original
        Spam.printNumInstances()
    printNumInstances = staticmethod(printNumInstances)

In [151]:
a = Sub()
b = Sub()
a.printNumInstances()

Extra stuff...
Number of instances: 5


In [152]:
Sub.printNumInstances()

Extra stuff...
Number of instances: 5


In [153]:
Spam.printNumInstances()

Number of instances: 5


In [154]:
class Other(Spam): pass # Inherit static method verbatim
c = Other()
c.printNumInstances()

Number of instances: 6


In [155]:
class Spam:
    numInstances = 0 # Use class method instead of static
    def __init__(self):
        Spam.numInstances += 1
    def printNumInstances(cls):
        print("Number of instances: %s" % cls.numInstances)
    printNumInstances = classmethod(printNumInstances)

In [156]:
a, b = Spam(), Spam()
a.printNumInstances()

Number of instances: 2


In [157]:
Spam.printNumInstances()

Number of instances: 2


In [158]:
class Spam:
    numInstances = 0 # Trace class passed in
    def __init__(self):
        Spam.numInstances += 1
    def printNumInstances(cls):
        print("Number of instances: %s %s" % (cls.numInstances, cls))
    printNumInstances = classmethod(printNumInstances)
class Sub(Spam):
    def printNumInstances(cls): # Override a class method
        print("Extra stuff...", cls) # But call back to original
        Spam.printNumInstances()
    printNumInstances = classmethod(printNumInstances)
class Other(Spam): pass

In [159]:
x = Sub()
y = Spam()
x.printNumInstances()

Extra stuff... <class '__main__.Sub'>
Number of instances: 2 <class '__main__.Spam'>


In [160]:
Sub.printNumInstances()

Extra stuff... <class '__main__.Sub'>
Number of instances: 2 <class '__main__.Spam'>


In [161]:
y.printNumInstances()

Number of instances: 2 <class '__main__.Spam'>


In [162]:
z = Other() # Call from lower sub's instance
z.printNumInstances()

Number of instances: 3 <class '__main__.Other'>


In [163]:
class Spam:
    numInstances = 0
    def count(cls): # Per-class instance counters
        cls.numInstances += 1 # cls is lowest class above instance
    def __init__(self):
        self.count() # Passes self.__class__ to count
    count = classmethod(count)
class Sub(Spam):
    numInstances = 0
    def __init__(self): # Redefines __init__
        Spam.__init__(self)
class Other(Spam): # Inherits __init__
    numInstances = 0

In [164]:
x = Spam()
y1, y2 = Sub(), Sub()

In [165]:
z1, z2, z3 = Other(), Other(), Other()
x.numInstances, y1.numInstances, z1.numInstances

(1, 2, 3)

In [166]:
Spam.numInstances, Sub.numInstances, Other.numInstances

(1, 2, 3)

# 9.7  Decorators and Metaclasses

In [167]:
class Spam:
    numInstances = 0
    def __init__(self):
        Spam.numInstances = Spam.numInstances + 1
    @staticmethod
    def printNumInstances():
        print("Number of instances created: %s" % Spam.numInstances)

In [168]:
a = Spam()
b = Spam()
c = Spam()
Spam.printNumInstances()

Number of instances created: 3


In [169]:
a.printNumInstances()

Number of instances created: 3


In [170]:
class Methods(object): # object needed in 2.X for property setters
    def imeth(self, x): # Normal instance method: passed a self
        print([self, x])
    @staticmethod
    def smeth(x): # Static: no instance passed
        print([x])
    @classmethod
    def cmeth(cls, x): # Class: gets class, not instance
        print([cls, x])
    @property # Property: computed on fetch
    def name(self):
        return 'Bob ' + self.__class__.__name__

In [171]:
obj = Methods()
obj.imeth(1)

[<__main__.Methods object at 0x036EEDB0>, 1]


In [172]:
obj.smeth(2)

[2]


In [173]:
obj.cmeth(3)

[<class '__main__.Methods'>, 3]


In [174]:
obj.name

'Bob Methods'

In [175]:
class tracer:
    def __init__(self, func): # Remember original, init counter
        self.calls = 0
        self.func = func
    def __call__(self, *args): # On later calls: add logic, run original
        self.calls += 1
        print('call %s to %s' % (self.calls, self.func.__name__))
        return self.func(*args)
@tracer # Same as spam = tracer(spam)
def spam(a, b, c): # Wrap spam in a decorator object
    return a + b + c
print(spam(1, 2, 3)) # Really calls the tracer wrapper object
print(spam('a', 'b', 'c')) # Invokes __call__ in class   

call 1 to spam
6
call 2 to spam
abc


In [176]:
def tracer(func): # Remember original
    def oncall(*args): # On later calls
        oncall.calls += 1
        print('call %s to %s' % (oncall.calls, func.__name__))
        return func(*args)
    oncall.calls = 0
    return oncall
class C:
    @tracer
    def spam(self,a, b, c): return a + b + c
x = C()
print(x.spam(1, 2, 3))
print(x.spam('a', 'b', 'c'))

call 1 to spam
6
call 2 to spam
abc


In [177]:
def decorator(cls): # On @ decoration
    class Proxy:
        def __init__(self, *args): # On instance creation: make a cls
            self.wrapped = cls(*args)
        def __getattr__(self, name): # On attribute fetch: extra ops here
            return getattr(self.wrapped, name)
    return Proxy

# 9.8 The super built-in function

In [178]:
class C: 
    def act(self):
        print('spam')
class D(C):
    def act(self):
        C.act(self) # Name superclass explicitly, pass self
        print('eggs')

In [179]:
X = D()
X.act()

spam
eggs


In [180]:
class C: 
    def act(self):
        print('spam')
class D(C):
    def act(self):
        super().act() # Reference superclass generically, omit self
        print('eggs')

In [181]:
X = D()
X.act()

spam
eggs


In [182]:
super

super

Ta thử khai báo hàm super()
```python
super()
```
`Output: RuntimeError: super(): no arguments`

In [183]:
class E(C):
    def method(self): # self is implicit in super...only!
        proxy = super() # This form has no meaning outside a method
        print(proxy) # Show the normally hidden proxy object
        proxy.act() # No arguments: implicitly calls superclass method!
E().method()

<super: <class 'E'>, <E object>>
spam


In [184]:
class A: 
    def act(self): print('A')
class B:
    def act(self): print('B')
class C(A):
    def act(self):
        super().act()

In [185]:
X = C()
X.act()

A


In [186]:
class C(B, A):
    def act(self):
        super().act() # If B is listed first, A.act() is no longer run!

In [187]:
X = C()
X.act()

B


In [188]:
class C(A, B): # Traditional form
    def act(self): # You probably need to be more explicit here
        A.act(self) # This form handles both single and multiple inher
        B.act(self) # And works the same in both Python 3.X and 2.X
X = C() # So why use the super() special case at all?
X.act()

A
B


In [189]:
class C:
    def __getitem__(self, ix): # Indexing overload method
        print('C index')
class D(C):
    def __getitem__(self, ix): # Redefine to extend here
        print('D index')
        C.__getitem__(self, ix) # Traditional call form works
        super().__getitem__(ix) # Direct name calls work too
        super()[ix]

In [190]:
X = C()
X[99]

C index


Gây ra lỗi ở lớp con 
```python
X = D()
X[99]
```
Output: 
```
D index
C index
C index
TypeError: 'super' object is not subscriptable
```

In [191]:
class X:
    def m(self): print('X.m')
class Y:
    def m(self): print('Y.m')
class C(X): # Start out inheriting from X
    def m(self): super().m()

In [192]:
i = C()
i.m()

X.m


In [193]:
C.__bases__ = (Y,) # Change superclass at runtime!
i.m()

Y.m


In [194]:
class C(X):
    def m(self): C.__bases__[0].m(self) # Special code for a special case
i = C()
i.m()

X.m


In [195]:
C.__bases__ = (Y,) # Same effect, without super()
i.m()

Y.m


In [196]:
class B:
    def __init__(self): print('B.__init__') # Disjoint class tree branches
class C:
    def __init__(self): print('C.__init__')
class D(B, C): pass

In [197]:
 x = D()

B.__init__


In [198]:
class D(B, C):
    def __init__(self): # Traditional form
        B.__init__(self) # Invoke supers by name
        C.__init__(self)

In [199]:
x = D()

B.__init__
C.__init__


In [200]:
class A:
    def __init__(self): print('A.__init__')
class B(A):
    def __init__(self): print('B.__init__'); A.__init__(self)
class C(A):
    def __init__(self): print('C.__init__'); A.__init__(self)

In [201]:
x = B()

B.__init__
A.__init__


In [202]:
 x = C() # Each super works by itsel

C.__init__
A.__init__


In [203]:
class D(B, C): pass # Still runs leftmost only
x = D()

B.__init__
A.__init__


In [204]:
class D(B, C):
    def __init__(self): # Traditional form
        B.__init__(self) # Invoke both supers by name
        C.__init__(self)

In [205]:
x = D()

B.__init__
A.__init__
C.__init__
A.__init__


In [206]:
class A:
    def __init__(self): print('A.__init__')
class B(A):
    def __init__(self): print('B.__init__'); super().__init__()
class C(A):
    def __init__(self): print('C.__init__'); super().__init__()

In [207]:
x = B()

B.__init__
A.__init__


In [208]:
x = C()

C.__init__
A.__init__


In [209]:
class D(B, C): pass
x = D()

B.__init__
C.__init__
A.__init__


In [210]:
 B.__mro__

(__main__.B, __main__.A, object)

In [211]:
 D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [212]:
class B:
    def __init__(self): print('B.__init__'); super().__init__()
class C:
    def __init__(self): print('C.__init__'); super().__init__()

In [213]:
x = B()

B.__init__


In [214]:
 x = C()

C.__init__


In [215]:
class D(B, C): pass # Inherits B.__init__ but B's MRO differs for D
x = D()

B.__init__
C.__init__


In [216]:
B.__mro__

(__main__.B, object)

In [217]:
D.__mro__

(__main__.D, __main__.B, __main__.C, object)

In [218]:
class B:
    def __init__(self): print('B.__init__')
class C:
    def __init__(self): print('C.__init__')
class D(B, C):
    def __init__(self): B.__init__(self); C.__init__(self)

In [219]:
x = D()

B.__init__
C.__init__


In [220]:
class B:
    def __init__(self): print('B.__init__'); super().__init__()
class C:
    def __init__(self): print('C.__init__'); super().__init__()
class D(B, C):
    def __init__(self): print('D.__init__'); super().__init__()
X = D()

D.__init__
B.__init__
C.__init__


In [221]:
 D.__mro__

(__main__.D, __main__.B, __main__.C, object)

In [222]:
class B:
    def __init__(self): print('B.__init__')
class D(B, C):
    def __init__(self): print('D.__init__'); super().__init__()
X=D()

D.__init__
B.__init__


In [223]:
class B:
    def __init__(self): print('B.__init__'); super().__init__()
class C:
    def __init__(self): print('C.__init__'); super().__init__()
class D(B, C):
    def __init__(self): print('D.__init__'); C.__init__(self); B.__init__(self)
X = D()

D.__init__
C.__init__
B.__init__
C.__init__


In [224]:
class A:
    def method(self): print('A.method'); super().method()
class B(A):
    def method(self): print('B.method'); super().method()
class C:
    def method(self): print('C.method') # No super: must anchor the chain!
class D(B, C):
    def method(self): print('D.method'); super().method()
X = D()
X.method()

D.method
B.method
A.method
C.method


In [225]:
class B(A):
    def method(self): print('B.method') # Drop super to replace A's method
class D(B, C):
    def method(self): print('D.method'); super().method()
X = D()
X.method()

D.method
B.method


In [226]:
class D(B, C):
    def method(self): print('D.method'); B.method(self); C.method(self)
D().method()

D.method
B.method
C.method


In [227]:
class A:
    def other(self): print('A.other')
class Mixin(A):
    def other(self): print('Mixin.other'); super().other()
class B:
    def method(self): print('B.method')
class C(Mixin, B):
    def method(self): print('C.method'); super().other(); super().method()

In [228]:
 C().method()

C.method
Mixin.other
A.other
B.method


In [229]:
C.__mro__

(__main__.C, __main__.Mixin, __main__.A, __main__.B, object)

In [230]:
class C(B, Mixin):
    def method(self): print('C.method'); super().other(); super().method()
C().method()

C.method
Mixin.other
A.other
B.method


In [231]:
C.__mro__

(__main__.C, __main__.B, __main__.Mixin, __main__.A, object)

In [232]:
class A:
    def other(self): print('A.other')
class Mixin(A):
    def other(self): print('Mixin.other'); super().other()
class B(A):
    def method(self): print('B.method')
class C(Mixin, B):
    def method(self): print('C.method'); super().other(); super().method()
C().method()

C.method
Mixin.other
A.other
B.method


In [233]:
C.__mro__

(__main__.C, __main__.Mixin, __main__.B, __main__.A, object)

In [234]:
class C(B, Mixin):
    def method(self): print('C.method'); super().other(); super().method()

In [235]:
 C().method()

C.method
Mixin.other
A.other
B.method


In [236]:
C.__mro__

(__main__.C, __main__.B, __main__.Mixin, __main__.A, object)

In [237]:
class C(Mixin, B):
    def method(self): print('C.method'); Mixin.other(self); B.method(self)

In [238]:
X = C()
X.method()

C.method
Mixin.other
A.other
B.method


In [239]:
class A:
    def method(self): print('A.method')
class Mixin(A):
    def method(self): print('Mixin.method'); super().method()
Mixin().method()

Mixin.method
A.method


In [240]:
class B(A):
    def method(self): print('B.method') # super here would invoke A after B
class C(Mixin, B):
    def method(self): print('C.method'); super().method()
C().method()

C.method
Mixin.method
B.method


In [241]:
class A:
    def method(self): print('A.method')
class Mixin(A):
    def method(self): print('Mixin.method'); A.method(self) # C irrelevant
class C(Mixin, B):
    def method(self): print('C.method'); Mixin.method(self)
C().method()

C.method
Mixin.method
A.method


In [242]:
class Employee:
    def __init__(self, name, salary): # Common superclass
        self.name = name
        self.salary = salary
class Chef1(Employee):
    def __init__(self, name): # Differing arguments
        Employee.__init__(self, name, 50000) # Dispatch by direct call
class Server1(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 40000)

In [243]:
bob = Chef1('Bob')
sue = Server1('Sue')
bob.salary, sue.salary

(50000, 40000)

In [244]:
class Chef2(Employee):
    def __init__(self, name):
        super().__init__(name, 50000) # Dispatch by super()
class Server2(Employee):
    def __init__(self, name):
        super().__init__(name, 40000)

In [245]:
bob = Chef2('Bob')
sue = Server2('Sue')
bob.salary, sue.salary

(50000, 40000)

In [246]:
class TwoJobs(Chef2, Server2): pass

Khi sử dụng đa thừa kế với super().
```python
tom = TwoJobs('Tom')
```
`Output: TypeError: __init__() takes 2 positional arguments but 3 were given`

In [247]:
TwoJobs.__mro__

(__main__.TwoJobs, __main__.Chef2, __main__.Server2, __main__.Employee, object)

In [248]:
 Chef2.__mro__

(__main__.Chef2, __main__.Employee, object)

In [249]:
class TwoJobs(Chef1, Server1): pass
tom = TwoJobs('Tom')
tom.salary

50000

In [250]:
class TwoJobs(Chef1, Server1):
    def __init__(self, name): Employee.__init__(self, name, 70000)
tom = TwoJobs('Tom')
tom.salary

70000

Đối với hàm super().
```python
class TwoJobs(Chef2, Server2):
    def __init__(self, name): super().__init__(name, 70000)
tom = TwoJobs('Tom')
```
`Output: TypeError: __init__() takes 2 positional arguments but 3 were given`

# 9.9 Class Gotchas

In [251]:
class X:
    a = 1 # Class attribute
I = X()
I.a # Inherited by instance

1

In [252]:
X.a

1

In [253]:
X.a = 2 # May change more than X
I.a

2

In [254]:
J = X() # J inherits from X's runtime values
J.a

2

In [255]:
class X: pass # Make a few attribute namespaces
class Y: pass

X.a = 1 # Use class attributes as variables
X.b = 2 # No instances anywhere to be found
X.c = 3
Y.a = X.a + X.b + X.c

for X.i in range(Y.a): print(X.i) # Prints 0..5

0
1
2
3
4
5


In [256]:
class Record: pass
X = Record()
X.name = 'bob'
X.job = 'Pizza maker'

In [257]:
class C:
    shared = [] # Class attribute
    def __init__(self):
        self.perobj = [] # Instance attribute

In [258]:
x = C() # Two instances
y = C() # Implicitly share class attrs
y.shared, y.perobj

([], [])

In [259]:
x.shared.append('spam') # Impacts y's view too!
x.perobj.append('spam') # Impacts x's data only
x.shared, x.perobj

(['spam'], ['spam'])

In [260]:
 y.shared, y.perobj

(['spam'], [])

In [261]:
C.shared # Stored on class and share

['spam']

In [262]:
x.shared.append('spam') # Changes shared object attached to class in place
x.shared = 'spam' # Changed or creates instance attribute attached to x

In [263]:
def generate():
    class Spam: # Spam is a name in generate's local scope
        count = 1
        def method(self):
            print(Spam.count) # Visible in generate's scope, per LEGB rule (E)
    return Spam()
generate().method()

1


In [264]:
def generate():
    return Spam()
class Spam: # Define at top level of module
    count = 1
    def method(self):
        print(Spam.count) # Works: in global (enclosing module)
generate().method()

1


In [265]:
def generate(label): # Returns a class instead of an instance
    class Spam:
        count = 1
        def method(self):
            print("%s=%s" % (label, Spam.count))
    return Spam

In [266]:
aclass = generate('Gotchas')
I = aclass()
I.method()

Gotchas=1


# Bài tập Excercise
## A. Quiz
**Câu 1:** Kể tên hai cách để mở rộng kiểu đối tượng built-in.

**Câu 2:** Lớp kiểu mới new-style là gì?

**Câu 3:** Lớp kiểu mới và lớp cổ điển khác nhau như thế nào?

**Câu 4:** Phương thức bình thường và phương thức tĩnh static khác nhau như thế nào?

**Câu 5:** Các công cụ như \_\_slots\_\_ và super có tác dụng ra sao trong mã của bạn?
## B. Coding

**Câu 6:** Thử nghiệm với lớp tập hợp  set mô tả trong “Các loại mở rộng bằng cách nhúng”. Chạy các lệnh để thực hiện các loại hoạt động sau:

+ Tạo hai tập hợp các số nguyên và tính intersection và union của chúng bằng cách sử dụng toán tử & và |.

+ Tạo một tập hợp từ một chuỗi và thử nghiệm index tập hợp của bạn. Cái mà các phương thức trong lớp được gọi là?

+ Hãy thử lặp lại các items trong tập hợp chuỗi của bạn bằng vòng lặp for. Cái mà phương thức chạy lần này?

+ Hãy thử tính toán intersection và union của tập hợp chuỗi của bạn và một chuỗi Python đơn giản. Nó có hoạt động không?

+ Bây giờ, hãy mở rộng tập hợp của bạn bằng cách phân lớp để xử lý nhiều toán hạng tùy ý bằng cách sử dụng đối số * args. Tính toán các itersection và union của nhiều toán hạng với lớp con set của bạn. Làm thế nào bạn có thể giao nhau ba hoặc nhiều tập hợp, cho rằng & có chỉ có hai mặt?

+ Bạn sẽ làm thế nào về việc mô phỏng các hoạt động danh sách khác trong lớp tập hợp? 
(Dấu:\_\_add\_\_ có thể bắt nối và \_\_getattr\_\_ có thể vượt qua hầu hết danh sách được đặt tên
các cuộc gọi phương thức như append vào danh sách được bao bọc.)

**Câu 7:** Phác thảo con vẹt. Xem xét cấu trúc nhúng đối tượng ở dưới

Viết mã tập hợp các lớp Python để triển khai cấu trúc này với composition. Mã 
đối tượng Scene của bạn để xác định một phương thức action và nhúng các instance của Customer, Clerk và Parrot (mỗi lớp sẽ xác định một phương thức line
in một thông điệp duy nhất). Các đối tượng nhúng có thể kế thừa từ một
lớp cha xác định line và chỉ đơn giản là cung cấp văn bản thông báo hoặc tự định nghĩa line. 

<img align='center' src='images/Parrot_Sketch.jpg'>

---
# <span style= 'color:blue'> Đáp án </span>
**1.** Bạn có thể nhúng một đối tượng built-in vào một lớp trình bao bọc  wrapper hoặc lớp con của kiểu built-in
trực tiếp. Cách tiếp cận thứ hai có xu hướng đơn giản hơn, vì hầu hết các hành vi ban đầu được tự động kế thừa.

**2.** Các lớp kiểu mới được viết bằng cách kế thừa từ lớp tích hợp đối tượng. Trong Python 3.X, tất cả các lớp đều tự động theo kiểu mới,trong 2.X các lớp kế thừa từ object này là kiểu mới và những lớp không có nó là classic.

**3.** Các lớp kiểu mới tìm kiếm mô hình kim cương diamond pattern của cây đa kế thừa một cách khác nhau, về cơ bản chúng tìm kiếm theo chiều rộng đầu tiên (trên), thay vì tìm kiếm theo chiều sâu (lên) trong cây kim cương. 

Các lớp kiểu mới cũng thay đổi kết quả của kiểu được tích hợp sẵn cho
phiên bản và lớp, không chạy các phương thức tìm nạp thuộc tính chung như \_\_get
attr\_\_ cho các phương pháp hoạt động built-in và hỗ trợ một bộ công cụ bổ sung nâng cao
bao gồm các property, descriptors, super và \_\_slots\_\_ danh sách thuộc tính của instance.

**4.** Các phương thức bình thường (instance) nhận được một đối số tự (instance ngụ ý) nhưng
phương thức tĩnh không. Phương thức tĩnh là các hàm đơn giản được lồng trong các đối tượng lớp.
Để làm cho một phương thức tĩnh, nó phải được chạy qua một hàm built-in đặc biệt
hoặc được trang trí bằng cú pháp decorator. 

Python 3.X cho phép các hàm số đơn giản trong một
lớp sẽ được gọi thông qua lớp mà không có bước này, nhưng gọi qua các instance
vẫn yêu cầu khai báo phương thức tĩnh.

**5.** Bạn không nên tự động sử dụng các công cụ nâng cao mà không cẩn thận
xem xét hàm ý của chúng. Các Slot, ví dụ, có thể phá vỡ mã; super có thể che giấu khi được sử dụng cho kế thừa đơn và trong đa kế thừa mang lại
với nó phức tạp đáng kể cho một trường hợp sử dụng riêng biệt; và cả hai đều yêu cầu phổ quát
triển khai để hữu ích nhất. 

Đánh giá các công cụ mới hoặc nâng cao là nhiệm vụ chính
của bất kỳ kỹ sư nào.

In [5]:
# 6
# File multiset.py
from setwrapper import Set

class MultiSet(Set):
    def intersect(self,*others):
        res=[]
        for x in self:
            for other in others:
                if x not in other: break
            else:
                res.append(x)
        return Set(res)
    
    def union(*args):
        res=[]
        for seq in args:
            for x in seq:
                if not x in res:
                    res.append(x)
        return Set(res)
    
x=MultiSet([1,2,3,4])
y=MultiSet([3,4,5])
z=MultiSet([0,1,2])
x&y,x|y

(Set:[3, 4], Set:[1, 2, 3, 4, 5])

In [6]:
x.intersect(y,z)

Set:[]

In [7]:
x.union(y,z)

Set:[1, 2, 3, 4, 5, 0]

In [9]:
w=MultiSet('spam')
w

Set:['s', 'p', 'a', 'm']

In [12]:
# 7
class Actor:
    def line(self):
        print(self.name+':',repr(self.says()))
    
class Customer(Actor):
    name='customer'
    def says(self):
        return "that's one ex-bird!"
    
class Clerk(Actor):
    name='clerk'
    def says(self):
        return "no it isn't..."
    
class Parrot(Actor):
    name='parrot'
    def says(self):
        return None

class Scece:
    def __init__(self):
        self.clerk=Clerk()          # Embed some instances
        self.customer=Customer()    # Scece is a composite
        self.subject=Parrot()
    
    def action(self):
        self.customer.line()
        self.clerk.line()
        self.subject.line()
        
Scece().action()

customer: "that's one ex-bird!"
clerk: "no it isn't..."
parrot: None
